In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [19]:
import os
from pathlib import Path
from tqdm import tqdm
import matplotlib.pyplot as plt
import random
from shutil import copyfile
from object_detection_fastai.helper.nms_center_distance import non_max_suppression_by_distance

In [4]:
from fastai import *
from fastai.vision import *
from fastai.callbacks import *
from fastai.callbacks.hooks import params_size

In [16]:
import openslide
import cv2
from torchvision import transforms

In [5]:
from exact_sync.v1.configuration import Configuration

In [6]:
path = Path('D:/Datasets/EIPH WSI/Pferd/Patches/train')
files = {p.name:p for p in Path("D:\\Datasets\\EIPH WSI\\Humane").glob("**/*.svs")}

learn = load_learner(path)

configuration = Configuration()
configuration.username = 'marzahl'
configuration.password = 'imagetagger'
configuration.host =  "https://exact.cs.fau.de/"


# Sync results with server

In [10]:
!pip install --upgrade EXCAT-Sync

  Attempting uninstall: EXCAT-Sync
    Found existing installation: EXCAT-Sync 0.0.17
    Uninstalling EXCAT-Sync-0.0.17:
      Successfully uninstalled EXCAT-Sync-0.0.17


In [7]:
from exact_sync.v1.api.annotations_api import AnnotationsApi
from exact_sync.v1.api.images_api import ImagesApi
from exact_sync.v1.api.image_sets_api import ImageSetsApi
from exact_sync.v1.api.annotation_types_api import AnnotationTypesApi
from exact_sync.v1.api.products_api import ProductsApi
from exact_sync.v1.api.teams_api import TeamsApi

from exact_sync.v1.models import ImageSet, Team, Product, AnnotationType, Image, Annotation, AnnotationMediaFile
from exact_sync.v1.rest import ApiException
from exact_sync.v1.api_client import ApiClient

In [8]:
client = ApiClient(configuration)

image_sets_api = ImageSetsApi(client)
annotations_api = AnnotationsApi(client)
annotation_types_api = AnnotationTypesApi(client)
images_api = ImagesApi(client)
product_api = ProductsApi(client)

In [9]:
image_sets = image_sets_api.list_image_sets(name__contains="Humane-EIPH")
image_sets.results

[{'creator': 1,
  'description': '',
  'id': 251,
  'images': [3627,
             3623,
             3622,
             3600,
             3601,
             3602,
             3603,
             3604,
             3605,
             3606,
             3607,
             3608,
             3609,
             3610,
             3611,
             3612,
             3613,
             3614,
             3615,
             3616,
             3617],
  'location': None,
  'main_annotation_type': 6,
  'name': 'Humane-EIPH',
  'path': 'imagetagger_4_251',
  'product_set': [3],
  'set_tags': [],
  'team': 4}]

In [13]:
images = {}
annotation_types = {}

for image_set in image_sets.results:
    for image in images_api.list_images(pagination=False, image_set=image_set.id).results:
        images[image.name] = image
    for product in image_set.product_set:
        for anno_type in annotation_types_api.list_annotation_types(product=product).results:
            annotation_types[anno_type.name] = anno_type

In [14]:
learn

{'2702_20 BB Human BAL-001.svs': {'annotations': 'Annotations not load please remove omit=annotations',
  'filename': '2702_20 BB Human BAL-001.svs',
  'height': 32360,
  'id': 3627,
  'image_set': 251,
  'image_type': 0,
  'mpp': 0.2533,
  'name': '2702_20 BB Human BAL-001.svs',
  'objective_power': 40.0,
  'time': datetime.datetime(2020, 8, 29, 12, 20, 18, 133678),
  'width': 34200}}

In [15]:
inference_results = Path('D:/Datasets/EIPH WSI/Humane/3627_human.p')
resultsArchive = pickle.load(open(str(inference_results),'rb'))

In [35]:
down_factor = 1
level = 0
nms_thresh = 40 
thresh = 0.35

mean, std = learn.data.stats

device = torch.device('cpu')
cpu_model = learn.model.to(device)

results = []
with torch.no_grad():
    
    for image_name in images:
        
        image = images[image_name]
        
        boxes = np.array(resultsArchive[image.name])
        boxes = boxes[boxes[:,5].argsort()]
        annos = non_max_suppression_by_distance(boxes, boxes[:, 5], nms_thresh)
        annos = annos[annos[:, 5] > thresh]
        
        for idx in tqdm(range(len(annos))):
            row = annos[idx]
            
            x_min = int(row[0]) 
            y_min = int(row[1]) 
            x_max = int(row[2]) 
            y_max = int(row[3])
            label = int(row[4])
            
            w, h = x_max - x_min, y_max - y_min
            
            if x_max - x_min < 10 or y_max - y_min < 10:
                continue
        
            deleted = False
            if w / h < 0.9 or w / h > 1.1:
                deleted = True
        
            vector = {"x1": x_min, "y1": y_min, "x2": x_max, "y2": y_max}
            annotation_type = annotation_types[str(label)]
            
            slide_path = files[image.name]
            slide = openslide.open_slide(str(slide_path))
            
            patch = np.array(slide.read_region(location=(int(x_min * down_factor), int(y_min * down_factor)),
                                                level=level, size=(w, h)))[:, :, :3]

            patch = cv2.resize(patch, (128, 128))
            patch = pil2tensor(patch / 255., np.float32)
            patch = transforms.Normalize(mean, std)(patch)

            score = cpu_model.eval()(patch[None, :, :, :])
        
            meta = {'Score': float(score)}
            
            annotation = Annotation(annotation_type=annotation_type.id, vector=vector, image=image.id, deleted=deleted, meta_data=meta)
            results.append(annotation)

100%|████████████████████████████████████████████████████████████████████████████| 13136/13136 [28:02<00:00,  7.81it/s]


In [37]:
for annotation in tqdm(results):
    annotations_api.create_annotation(body=annotation)
    

100%|████████████████████████████████████████████████████████████████████████████| 13136/13136 [43:26<00:00,  5.04it/s]
